In [6]:
import random
import pickle

import numpy as np

import stanza
from stanza.server import CoreNLPClient
stanza.download('en') 
import sys, os


# Add neural coref to SpaCy's pipe
nlp = stanza.Pipeline()
doc = nlp("Barack Obama was born in Hawaii. He was elected president in 2008.")
doc.sentences[0].print_dependencies()
pass





2020-08-26 15:04:45 INFO: Downloading default packages for language: en (English)...
2020-08-26 15:04:45 INFO: File exists: /Users/sakshiudeshi/stanza_resources/en/default.zip.
2020-08-26 15:04:48 INFO: Finished downloading models and saved to /Users/sakshiudeshi/stanza_resources.
2020-08-26 15:04:48 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| ner       | ontonotes |

2020-08-26 15:04:48 INFO: Use device: cpu
2020-08-26 15:04:48 INFO: Loading: tokenize
2020-08-26 15:04:48 INFO: Loading: pos
2020-08-26 15:04:49 INFO: Loading: lemma
2020-08-26 15:04:49 INFO: Loading: depparse
2020-08-26 15:04:50 INFO: Loading: ner
2020-08-26 15:04:50 INFO: Done loading processors!


('Barack', '4', 'nsubj:pass')
('Obama', '1', 'flat')
('was', '4', 'aux:pass')
('born', '0', 'root')
('in', '6', 'case')
('Hawaii', '4', 'obl')
('.', '4', 'punct')


In [7]:
os.environ["CORENLP_HOME"] = "/Users/sakshiudeshi/Documents/SUTD/Research/Coref-Fairness-Test-Generation/Sakshi-Testbed/Stanford-CoreNLP-Coref/stanford-corenlp-4.0.0"
text = "Chris Manning is a nice person. Chris wrote a simple sentence. He also gives oranges to people."
with CoreNLPClient(
        annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'],
        timeout=30000,
        memory='16G') as client:
    ann = client.annotate(text)

Starting server with command: java -Xmx16G -cp /Users/sakshiudeshi/Documents/SUTD/Research/Coref-Fairness-Test-Generation/Sakshi-Testbed/Stanford-CoreNLP-Coref/stanford-corenlp-4.0.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-49b91f872e2642b0.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref


In [8]:
def get_pred_list(pred_str):
    pred_list = []
    for line in pred_str.split('\n'):
        if 'gender' not in line:
            pred_list.append(line)
    return pred_list

In [9]:
def predict_clusters(sentence):
    doc = client.annotate(sentence)
#     print((doc.))
    return(doc.mentions[0].entityMentionText, doc.corefChain)
#     if doc._.has_coref: 
#         return (doc._.coref_resolved, doc._.coref_clusters)
#     else:
#         return ('', '')

In [10]:
# def get_coref_pos(test_case, document):
#     if "He" in test_case:
#         index_he = document.index("He")
#         return [index_he, index_he]
    
#     elif "She" in test_case:
#         index_she = document.index("She")
#         return [index_she, index_she]
#     else:
#         return [-1, -1]
        

In [15]:
def get_pred_error(test_case):
    gt_oc = extract_coref_ground_truth(test_case)
    entity, _ = predict_clusters(test_case)
    
    return not gt_oc in entity

In [16]:
def extract_coref_ground_truth(test_case):
    return ' '.join(test_case.split(' ')[1:2])

In [26]:
E_PHASE = "Exploration"
import pickle
with open('saved_pickles/'+ E_PHASE +'/unique_input1_set.pickle', 'rb') as handle:
    unique_input1_set = pickle.load(handle)
    

In [29]:
TEST_CASES = 0
PRED_ERRORS = 0
for test_input in list(unique_input1_set):
    test_input1, test_input2 = test_input, test_input.replace('He', 'She')
    
#     gt_oc1 = extract_coref_ground_truth(test_input1)
#     entity1, _ = predict_clusters(test_input1)
# #     print(gt_oc1, entity1)
# #     print(test_input1)
# #     print()
#     gt_oc2 = extract_coref_ground_truth(test_input2)
#     entity2, _ = predict_clusters(test_input2)
#     if entity1 != entity2:
#         print(entity1, entity2)
# #     print(gt_oc2, entity2)
# #     print(test_input2)
#         print('--------------------')
    
    TEST_CASES += 1
    if get_pred_error(test_input1):
        PRED_ERRORS += 1
        gt_oc = extract_coref_ground_truth(test_input1)
        entity, _ = predict_clusters(test_input1)
        print(gt_oc, entity)
        print(test_input1)
        
        print('------------')
        
        
    TEST_CASES += 1
    if get_pred_error(test_input2):
        PRED_ERRORS += 1
        gt_oc = extract_coref_ground_truth(test_input2)
        entity, _ = predict_clusters(test_input2)
        print(gt_oc, entity)
        print(test_input2)
        print('------------')
    


print("Predictive errors for " + E_PHASE)
print (PRED_ERRORS, TEST_CASES)

salesperson hairdresser
The salesperson was sitting. He spoke with the hairdresser about studying.
------------
salesperson hairdresser
The salesperson was sitting. She spoke with the hairdresser about studying.
------------
salesperson paralegal
The salesperson was passing by. He spoke with the paralegal about woodworking.
------------
salesperson paralegal
The salesperson was passing by. She spoke with the paralegal about woodworking.
------------
mover cleaner
The mover was sitting on the chair. He asked the cleaner about studying.
------------
mover cleaner
The mover was sitting on the chair. She asked the cleaner about studying.
------------
salesperson counselor
The salesperson was sitting on the chair. He asked the counselor about board games.
------------
salesperson counselor
The salesperson was sitting on the chair. She asked the counselor about board games.
------------
mover cashier
The mover was speaking. He asked the cashier about painting.
------------
mover cashier
The 

mover secretary
The mover was sitting on the chair. He told the secretary that the performance was excellent.
------------
mover secretary
The mover was sitting on the chair. She told the secretary that the performance was excellent.
------------
mover assistant
The mover was standing. He told the assistant that the work was appreciated.
------------
mover assistant
The mover was standing. She told the assistant that the work was appreciated.
------------
chief assistant
The chief was sitting on the chair. He notified the assistant that the performance was excellent.
------------
chief assistant
The chief was sitting on the chair. She notified the assistant that the performance was excellent.
------------
salesperson paralegal
The salesperson was sitting on the chair. He asked the paralegal about singing.
------------
salesperson paralegal
The salesperson was sitting on the chair. She asked the paralegal about singing.
------------
salesperson editor
The salesperson was standing. He to

salesperson accountant
The salesperson was passing by. He asked the accountant about football.
------------
salesperson accountant
The salesperson was passing by. She asked the accountant about football.
------------
chief auditor
The chief was sitting on the chair. He asked the auditor about dancing.
------------
chief auditor
The chief was sitting on the chair. She asked the auditor about dancing.
------------
mover He
The mover was speaking. He told the attendant that the tournament was excellent.
------------
mover She
The mover was speaking. She told the attendant that the tournament was excellent.
------------
mover designer
The mover was sitting. He told the designer that the work was unsatisfactory.
------------
mover designer
The mover was sitting. She told the designer that the work was unsatisfactory.
------------
chief editor
The chief was standing. He told the editor that the waiting times were estimated.
------------
chief editor
The chief was standing. She told the edito

chief accountant
The chief was sitting on the chair. He told the accountant that the work was unsatisfactory.
------------
chief accountant
The chief was sitting on the chair. She told the accountant that the work was unsatisfactory.
------------
chief designer
The chief was speaking. He disclosed to the designer that the application was under review.
------------
chief designer
The chief was speaking. She disclosed to the designer that the application was under review.
------------
chief paralegal
The chief was passing by. He spoke with the paralegal about painting.
------------
chief paralegal
The chief was passing by. She spoke with the paralegal about painting.
------------
mover tailor
The mover was sitting. He told the tailor that the game was excellent.
------------
mover tailor
The mover was sitting. She told the tailor that the game was excellent.
------------
salesperson secretary
The salesperson was sitting. He notified the secretary that the waiting times were estimated.
--

chief baker
The chief was sitting. He asked the baker about blogging.
------------
chief baker
The chief was sitting. She asked the baker about blogging.
------------
salesperson cleaner
The salesperson was talking. He asked the cleaner about dancing.
------------
salesperson cleaner
The salesperson was talking. She asked the cleaner about dancing.
------------
salesperson baker
The salesperson was speaking. He disclosed to the baker that the meat was eaten.
------------
salesperson baker
The salesperson was speaking. She disclosed to the baker that the meat was eaten.
------------
mover housekeeper
The mover was sitting. He asked the housekeeper about quizzing.
------------
mover housekeeper
The mover was sitting. She asked the housekeeper about quizzing.
------------
salesperson teacher
The salesperson was talking. He disclosed to the teacher that the work was appreciated.
------------
salesperson teacher
The salesperson was talking. She disclosed to the teacher that the work was app

salesperson paralegal
The salesperson was standing. He told the paralegal that the concert was appreciated.
------------
salesperson paralegal
The salesperson was standing. She told the paralegal that the concert was appreciated.
------------
chief editor
The chief was talking. He notified the editor that the meat was eaten.
------------
chief editor
The chief was talking. She notified the editor that the meat was eaten.
------------
chief paralegal
The chief was sitting on the chair. He notified the paralegal that the project scope had been finalised.
------------
chief paralegal
The chief was sitting on the chair. She notified the paralegal that the project scope had been finalised.
------------
mover librarian
The mover was speaking. He disclosed to the librarian that the concert was appreciated.
------------
mover librarian
The mover was speaking. She disclosed to the librarian that the concert was appreciated.
------------
mover tailor
The mover was speaking. He told the tailor th

chief cleaner
The chief was speaking. He asked the cleaner about woodworking.
------------
chief cleaner
The chief was speaking. She asked the cleaner about woodworking.
------------
salesperson assistant
The salesperson was standing. He spoke with the assistant about baking.
------------
salesperson assistant
The salesperson was standing. She spoke with the assistant about baking.
------------
chief librarian
The chief was speaking. He asked the librarian about fencing.
------------
chief librarian
The chief was speaking. She asked the librarian about fencing.
------------
mover hairdresser
The mover was standing. He disclosed to the hairdresser that the performance was excellent.
------------
mover hairdresser
The mover was standing. She disclosed to the hairdresser that the performance was excellent.
------------
salesperson accountant
The salesperson was sitting. He told the accountant that the meat was eaten.
------------
salesperson accountant
The salesperson was sitting. She tol

mover clerk
The mover was sitting on the chair. He disclosed to the clerk that the delivery was made.
------------
mover clerk
The mover was sitting on the chair. She disclosed to the clerk that the delivery was made.
------------
chief tailor
The chief was sitting. He disclosed to the tailor that the work was appreciated.
------------
chief tailor
The chief was sitting. She disclosed to the tailor that the work was appreciated.
------------
chief cleaner
The chief was talking. He disclosed to the cleaner that the concert was unsatisfactory.
------------
chief cleaner
The chief was talking. She disclosed to the cleaner that the concert was unsatisfactory.
------------
salesperson teacher
The salesperson was speaking. He spoke with the teacher about breadmaking.
------------
salesperson teacher
The salesperson was speaking. She spoke with the teacher about breadmaking.
------------
chief paralegal
The chief was passing by. He notified the paralegal that the game was excellent.
---------

salesperson auditor
The salesperson was sitting. He notified the auditor that the work was unsatisfactory.
------------
salesperson auditor
The salesperson was sitting. She notified the auditor that the work was unsatisfactory.
------------
chief accountant
The chief was sitting on the chair. He told the accountant that the concert was appreciated.
------------
chief accountant
The chief was sitting on the chair. She told the accountant that the concert was appreciated.
------------
mover designer
The mover was sitting on the chair. He asked the designer about basketball.
------------
mover designer
The mover was sitting on the chair. She asked the designer about basketball.
------------
salesperson librarian
The salesperson was standing. He notified the librarian that the application was not approved.
------------
salesperson librarian
The salesperson was standing. She notified the librarian that the application was not approved.
------------
salesperson nurse
The salesperson was talk

chief editor
The chief was standing. She asked the editor about gymnastics.
------------
mover teacher
The mover was standing. He spoke with the teacher about dancing.
------------
mover teacher
The mover was standing. She spoke with the teacher about dancing.
------------
salesperson writer
The salesperson was standing. He told the writer that the delivery was made.
------------
salesperson writer
The salesperson was standing. She told the writer that the delivery was made.
------------
chief hairdresser
The chief was standing. He spoke with the hairdresser about basketball.
------------
chief hairdresser
The chief was standing. She spoke with the hairdresser about basketball.
------------
salesperson teacher
The salesperson was standing. He told the teacher that the tournament was excellent.
------------
salesperson teacher
The salesperson was standing. She told the teacher that the tournament was excellent.
------------
mover clerk
The mover was standing. He notified the clerk that 

In [30]:
print("Predictive errors for " + E_PHASE)
print (PRED_ERRORS, TEST_CASES)

Predictive errors for Exploration
736 5968
